In [14]:
# Insert all parameters here. Note that this is the only place where a user inputs can be processed.

# Altergo parameters
factory_api = "https://altergo.io/"
iot_api = "https://iot.altergo.io/" 
api_key = "" # Note that your API key will become public when it is injected

# Setup parameters
asset_sn = "ISO-ENTSO-E_Belgium_BE" # Get the target input ISO asset serial number
target_asset = ["PRED-ALFN-BANK-001-01","PRED-ALFN-BANK-001-02","PRED-ALFN-BANK-001-03"] #Get the target list of assets 
input_sn_list = ["DALMP"] #Get the input sensor list
sn_list = ["SOC_dispatch","E_out_DAM","E_in_DAM","Revenue Generated","Total Revenue","Charge Cost","Profit Sum","Revenue Sum"] # Get the target list of sensors
start_time = "2021-9-1 00:00" # Get the start time for calculating: Expect as YYYY-MM-DD hh:mm:ss
end_time = "2021-9-8 00:00" # Get the end time for calculating : Expect as YYYY-MM-DD hh:mm:ss

# Model Parameters
initial_SoC = "55" # Get the model parameter

# Update flag
should_update = False

In [15]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import json
from scipy import stats
from typing import Union
from copy import deepcopy
from getpass import getpass

In [18]:
# Input cleanup and setup

os.environ["ALTERGO_FACTORY_API"] = factory_api
os.environ["ALTERGO_IOT_API"] = iot_api

start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
start_time = start_time.timestamp() * 1000
end_time = end_time.timestamp() * 1000

In [25]:
!pip install -qqq -U pyomo
!apt-get install -y -qq glpk-utils
!pip install -qqq -U git+https://bitbucket.org/freemens/ion_sdk.git@master
!apt-get install -y -qq glpk-utils
clear_output()

In [24]:
from pyomo.environ import *
from ion_sdk.edison_api.models.factoryModel import EdisonGenericComponent, Model,CurrentState
from ion_sdk.edison_api.edison_api import Client, dataUpdateMethod, edisonDate
from ion_sdk.edison_api.models.factory import getParameterValue, getSensorCodeByName, EdisonComponent
from ion_sdk.tools.toolbox import decimate_data

ModuleNotFoundError: No module named 'ion_sdk'

In [ ]:
edApi = Client(api_key) # Connect to the ALTG Client
asset = edApi.getAsset(asset_sn) # Get the Asset from ALTG
edApi.getAssetDataFrame([asset], input_sn_list, start_time, end_time) # Get the asset dataframe from ALTG
energyCapacity = getParameterValue(target_asset[0].model,'Energy','Nom') # Get the capacity of the battery
pmax = getParameterValue(target_asset[0].model,'Energy','Nom') # Get the capacity of the battery
nCycle =getParameterValue(target_asset[0].model,'nCycles per day','Nom')
eff =getParameterValue(target_asset[0].model,'Round Trip Efficiency','Nom')/100.0

In [ ]:
# This cell will vary from user to user, client to client
# First, (optionally) remap the columns
asset.df.rename(columns=sn_mapping, inplace=True)
# Add any other adjustment steps in this cell
initial_SoC = int(initial_SoC)
asset.df.reset_index(inplace=True)
asset.df['time_stamp'] = asset.df.date
asset.df['time_stamp'] = pd.to_datetime(asset.df['time_stamp'])

for i in range(0,asset.df.index.iloc[-1]):
   asset.df["time_stamp"].iloc[i] = asset.df["time_stamp"].iloc[i].replace(second=0, microsecond=0, minute=0, hour=asset.df["time_stamp"].iloc[i].hour)+timedelta(hours=asset.df["time_stamp"].iloc[i].minute//30)

asset.df['hour'] = asset.df.time_stamp.index
asset.df['Unnamed1: 0'] = asset.df['hour']
asset.df.insert(0, 'Unnamed: 0', asset.df['Unnamed1: 0'] )
asset.df.drop(["Unnamed1: 0","time_stamp"],axis=1,inplace=True)
asset.df = asset.df.rename(columns={'date': 'time_stamp','DALMP': 'lbmp'})
df1 = asset.df
df2 = asset.df

In [ ]:
# Set up the models
#@title 7 Day Average (Run time: < 5s)
def dispatch_strategy_1(df, energy_capacity, p_max, n_cycles, eff, initial_Soc, soc_low, soc_high):
    """
    Strategy 1: Take a mean for the day ahead. If price is higher than average, 
    discharge. If price is lower than average, charge. Do this while satisfying 
    all other constraints (like number of cycles in a day, SOC constraints, 
    etc.)

    """

    average_price = df["lbmp"].mean()
    df = df.reset_index()
    df["SOC"] = 0
    df.loc[0, "SOC"] = initial_Soc
    df["E_in"] = 0
    df["E_out"] = 0

    total_throughput = 0
    max_throughput = energy_capacity * n_cycles

    for t in range(1, df.index.size):
        if t % 24 == 0:
            total_throughput = 0
        
        df.loc[t, "SOC"] = df.loc[t-1,"SOC"]
        if df.loc[t,"lbmp"] > average_price:
            if df.loc[t-1, "SOC"] - p_max/eff/energy_capacity >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max
            elif df.loc[t-1, "SOC"] - p_max/eff/energy_capacity/2 >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max/2
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max/2

        else:
            if df.loc[t-1,"SOC"] + p_max * eff / energy_capacity <= soc_high:
                df.loc[t,"E_in"] = p_max
                df.loc[t, "SOC"] = df.loc[t-1, "SOC"] + df.loc[t,"E_in"] * eff / energy_capacity

    return df.set_index("time_stamp")

target_asset[0].df = dispatch_strategy_1(df1, energyCapacity, pmax, nCycle, eff, (initial_SoC/100.0), 0.05, 0.95)

In [29]:
#@title Hourly Average (Run time: < 5s)
def dispatch_strategy_2(df, energy_capacity, p_max, n_cycles, eff, initial_Soc, soc_low, soc_high):
    """
    Take average across 7 days for each hour. If the price for that hour is 
    greater than the average price for that hour, discharge. If the price for 
    that hour is less than the average price for that hour, charge. Also ensure 
    that all other constraints are met (throughput, SOC, etc.)
    """

    df = df.reset_index()
    df["SOC"] = 0
    df.loc[0, "SOC"] = initial_Soc
    df["E_in"] = 0
    df["E_out"] = 0

    total_throughput = 0
    max_throughput = energy_capacity * n_cycles

    df.index = df.index % 24 # Convert to 24 hour indices. Makes it easier for taking averages.
    average_price_df = pd.DataFrame({"Average LMP" : [0 for _ in range(0, 24)]}, index = range(0, 24))
    for i in range(0, 24):
        average_price_df.loc[i, "Average LMP"] = df.loc[i,"lbmp"].mean()

    df = df.reset_index(drop = True)
    for t in range(1, df.index.size):
        if t % 24 == 0:
            total_throughput = 0

        df.loc[t, "SOC"] = df.loc[t-1,"SOC"]
        if df.loc[t,"lbmp"] > average_price_df.loc[t % 24, "Average LMP"]:
            if df.loc[t-1, "SOC"] - p_max / eff / energy_capacity >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max
            elif df.loc[t-1, "SOC"] - p_max/eff/energy_capacity/2 >= soc_low:
                if total_throughput < max_throughput:
                    df.loc[t,"E_out"] = p_max/2
                    df.loc[t, "SOC"] = df.loc[t-1, "SOC"] - df.loc[t,"E_out"]/eff/energy_capacity
                    total_throughput += p_max/2

        else:
            if df.loc[t-1,"SOC"] + p_max * eff / energy_capacity <= soc_high:
                df.loc[t,"E_in"] = p_max
                df.loc[t, "SOC"] = df.loc[t-1, "SOC"] + df.loc[t,"E_in"] * eff / energy_capacity

    return df.set_index("time_stamp")

target_asset[1].df = dispatch_strategy_2(df2, energyCapacity, pmax, nCycle, eff, (initial_SoC/100.0), 0.05, 0.95)

NameError: name 'df2' is not defined

In [19]:
#@title Scheduling Optimization Strategy: DA (Run time: <1 minute)

def df_conversion_Arb(model, start_hour, end_hour):
     """
     Parameters
     model : pyomo model
         Model that has been solved
    
     Returns
     dataframe
        
    """
     hours = range(model.T[start_hour + 1], model.T[end_hour + 1] + 1)
     E_in = [value(model.E_in[i]) for i in hours]
     E_out = [value(model.E_out[i]) for i in hours]
     lmp = [model.Price.extract_values()[None][i] for i in hours]
     state_of_charge = [value(model.S[i])/model.Capacity * 100 for i in hours]

     model_dict = dict(
         hour=hours,
         E_in=E_in,
         E_out=E_out,
         lmp=lmp,
         state_of_charge=state_of_charge
     )

     df = pd.DataFrame(model_dict)

     return df


def optimize_Arb(df, start_hour=0, end_hour=23,energyCapacity=2.0,pmax=1,nCycle=1.0,eff=0.9,initial_SoC=50):
     """    
     Parameters
     df : dataframe
         dataframe with columns of hourly LMP and the hour of the year
     start_hour : optional
         Set the first hour of the year to be considered in the optimization
     end_hour : optional
         Set the last hour of the year to be considered in the optimization
    
     Returns
     dataframe
         hourly state of charge, charge/discharge behavior, lmp
     """

     #Filter the data
     df = df.loc[start_hour:end_hour, :]
     print(df)
     model = ConcreteModel()

     # Define model parameters  # To be fetched from User Inputs
     model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)
     model.Pmax = Param(initialize=pmax,doc='Max rate of power flow (kW) in or out')
     model.Capacity = Param(initialize=energyCapacity, doc='Max storage (kWh)')
     model.Cycle = nCycle 
     energyThroughput = model.Cycle * model.Capacity
     model.Dthroughputmax = Param(initialize=energyThroughput, doc='Max discharge in 24 hour period')
     model.Price = Param(initialize=df.lbmp.tolist(), within=Any, doc='LMP for each hour')
     eff = eff # Round trip storage efficiency

     # Charge, discharge, and state of charge
     model.E_in = Var(model.T, domain=NonNegativeReals)
     model.E_out = Var(model.T, domain=NonNegativeReals)
     model.S = Var(model.T, bounds=(0, model.Capacity))

     #Set all constraints
     def storage_state(model, t):
         'Storage changes with flows in/out and efficiency losses'
         # Set first hour state of charge to half of max
         if t == model.T.first():  
             return model.S[t] == model.Capacity * (initial_SoC/100.0)   #User Input Form: To be finalized  
         else:
             return (model.S[t] == (model.S[t-1] 
                                 + (model.E_in[t-1] * np.sqrt(eff)) 
                                 - (model.E_out[t-1] / np.sqrt(eff))))

     model.charge_state = Constraint(model.T, rule=storage_state)

     def discharge_constraint(model, t):
         "Maximum dischage within a single hour"
         return (model.E_out[t]) <= model.Pmax

     model.discharge = Constraint(model.T, rule=discharge_constraint)  

     def charge_constraint(model, t):
         "Maximum charge within a single hour"
         return (model.E_in[t]) <= model.Pmax

     model.charge = Constraint(model.T, rule=charge_constraint)

     def current_charge(model, t):
         'Limit discharge to the amount of charge in battery, including losses'
         return model.E_out[t] <= model.S[t] / np.sqrt(eff)
     model.current_charge = Constraint(model.T, rule=current_charge)
    
     def dthroughput_limit(model, t):
         "Limit on discharge within a 24 hour period : Daily discharge throughput"
         max_t = model.T.last()
         if t < max_t - 24:
            return sum(model.E_out[i]  for i in range(start_hour,(start_hour+24))) <= model.Dthroughputmax
         else:
            return Constraint.Skip
         
     model.limit_out = Constraint(model.T, rule=dthroughput_limit)      
    
     income = sum(df.loc[t, 'lbmp'] * model.E_out[t] for t in model.T)
     expenses = sum(df.loc[t, 'lbmp'] * model.E_in[t] for t in model.T)     #confirmation
     profit = income - expenses
     model.objective = Objective(expr=profit, sense=maximize)

     # Solve the model
     SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

     results = df_conversion_Arb(model, start_hour=start_hour,
                              end_hour=end_hour)
     results['time_stamp'] = df.loc[:, 'time_stamp']

     return results

target_asset[2].df = optimize_Arb(asset.df,asset.df.hour.iloc[0],167,energyCapacity,pmax,nCycle,eff,50)


NameError: name 'asset' is not defined

In [17]:
# Return data to ALTG

# This cell will vary from user to user, client to client.
# Depending on the architecture, the client will have a set sensor to which data needs to be returns for State of Health

uploadSensorList = ["SOC_dispatch", "E_out_DAM","E_in_DAM","Revenue Generated","Total Revenue","Charge Cost","Profit Sum","Revenue Sum"]

flag = 0
for a in target_asset:
    for x in uploadSensorList:
        if x not in a.df.columns:
            flag = 1    
assert flag == 0

edApi.updateSensorDataByFile(asset1, uploadSensorList)
edApi.updateSensorDataByFile(asset2, uploadSensorList)
edApi.updateSensorDataByFile(asset3, uploadSensorList)
print("Successfully Uploaded Data to Edison")

NameError: name 'pd' is not defined

In [ ]:
# {
#   "factory_api": "https://altergo.io/",
#   "iot_api": "https://iot.altergo.io/",
#   "api_key": "",
#   "asset_sn": "",
#   "sn_list": [],
#   "sn_mapping": {},
#   "start_time": "",
#   "end_time": "",
#   "params_cal": {
#     "Ea": 77172.17070551902,
#     "z": 0.65,
#     "k": [
#       0.03540461,
#       1
#     ],
#     "alpha": null,
#     "integrate": null
#   },
#   "params_cyc": {
#     "alpha": null,
#     "k": [
#       [
#         0.00575232,
#         -0.01347511,
#         -0.01221794,
#         0.00464791,
#         0.01542588,
#         0.00143808,
#         -0.0059255,
#         -0.0009856
#       ],
#       [
#         0.0003,
#         -0.000074820516,
#         -0.000130873528,
#         0.00092049,
#         -0.0008841,
#         -0.00011354,
#         -0.00435159,
#         0.00320431
#       ]
#     ],
#     "Ea": 0,
#     "z": null
#   },
#   "cal_estimate": {
#     "error": "warn",
#     "init_SoH": 1.0
#   },
#   "cyc_estimate": {
#     "init_SoH": 1.0,
#     "threshold": 0.1, 
#     "error": "warn"
#   },
#   "bbusr": "jay-ion",
#   "bbkey": "albdhgor299gns84",
#   "should_update": true,
# }